First we scrape the Wikipedia page and create a dataframe with the table.

In [33]:
import requests
import pandas as pd

r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
dataframes=pd.read_html(r.text, header=0)
df = dataframes[0]

After that we remove all the rows with unassigned boroughs and reset the indexes because of the removed data.

In [34]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df = df.reset_index().drop('index', axis=1)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Now we assign the boroughs to the unassigned neighborhoods (at the time I scraped the data there were no rows where this happened but anyways I'm adding it in case the data changes in the future).

In [35]:
df[(df['Borough'] != 'Not assigned') & (df['Neighborhood'] == 'Not assigned')]['Neighborhood'] = df[(df['Borough'] != 'Not assigned') & (df['Neighborhood'] == 'Not assigned')]['Borough']

We check that there's no duplicated postal codes left

In [36]:
len(df['Postal Code'].unique()) == len(df)

True

We print the dataframe shape

In [37]:
df.shape

(103, 3)

Now we download the csv file with latitudes and longitudes

In [25]:
df_lt_lg = pd.read_csv('http://cocl.us/Geospatial_data')
df_lt_lg.sort_values(by=['Postal Code'])
df_lt_lg.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We merge both dataframes by the 'Postal Code' column

In [26]:
df.sort_values(by=['Postal Code'])
df = pd.merge(df, df_lt_lg, on='Postal Code')

And we have our merged dataframe

In [32]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
